In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import csv
import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)

pd.options.display.max_rows = 1000


files = ['obama']

In [10]:
def rl_parse():
    data = {}
    for file in files:
        log.info("parsing file: ../data/{}-openie.e2rdf".format(file))
        my_cols = ["Confidence", "Context", "Subject", "Predicate", "Object", "Other_arg", "Temporal", "Spatial", "Sentence_id", "Sentence"]
        values = pd.read_csv("../data/{}-openie.e2rdf".format(file), names=my_cols, sep='\t', quoting=csv.QUOTE_NONE)
        values.drop(values.columns[[0, 1, 8]], axis=1, inplace=True)
        data[file] = values
    return data

In [11]:
data = rl_parse()

INFO:root:parsing file: ../data/obama-openie.e2rdf


In [12]:
data['obama']

,Subject,Predicate,Object,Other_arg,Temporal,Spatial,Sentence
0,seɪn ɵ'bɑːmə,born,August 4,NaN,NaN,NaN,Barack Hussein Obama II (US /bəˈrɑːk huːˈseɪn ...
1,Obama,is a graduate of,Columbia University,NaN,NaN,NaN,"Born in Honolulu, Hawaii, Obama is a graduate ..."
2,He,before earning,his law degree,NaN,NaN,NaN,He was a community organizer in Chicago before...
3,He,taught,constitutional law,from 1992 to 2004,NaN,at University of Chicago Law School,He worked as a civil rights attorney and taugh...
4,Obama,received,national attention,NaN,during his campaign | In 2004,NaN,"In 2004, Obama received national attention dur..."
5,He,was inaugurated,as president,NaN,on January 20,NaN,He then defeated Republican nominee John McCai...
6,the Democratic Party,lost,a total of 63 seats,NaN,after a lengthy debate over federal spending a...,NaN,"In January 2011, the Republicans regained cont..."
7,Obama,was reelected,president,NaN,in November 2012,NaN,Obama was reelected president in November 2012...
8,Mitt Romney,[is] nominee [of],Republican,NaN,NaN,NaN,Obama was reelected president in November 2012...
9,domestic policies,related,to gun control,in response to the Sandy Hook Elementary Schoo...,NaN,NaN,"During his second term, Obama has promoted dom..."
